<a href="https://colab.research.google.com/github/salmaan-azam/Product-Price-Prediction/blob/main/Mercari(Feature_Engineering_and_Modelling).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# Importing Libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
# Loading the data
train = pd.read_csv('gdrive/My Drive/Mercari/train.tsv',delimiter='\t', engine='c')
test1 = pd.read_csv('gdrive/My Drive/Mercari/test.tsv',delimiter='\t', engine='c')
test2 = pd.read_csv('gdrive/My Drive/Mercari/test_stg2.tsv',delimiter='\t', engine='c')

In [ ]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [ ]:
# How many duplicates are present in train and test data ?
print("Number of duplicates in train : {}".format(sum(train.duplicated())))
print("Number of duplicates in test1 : {}".format(sum(test1.duplicated())))
print("Number of duplicates in test2 : {}".format(sum(test2.duplicated())))

Number of duplicates in train : 0
Number of duplicates in test1 : 0
Number of duplicates in test2 : 0


In [ ]:
# Is there any null values are present in train and test data?
train.isnull().any()

train_id             False
name                 False
item_condition_id    False
category_name         True
brand_name            True
price                False
shipping             False
item_description      True
dtype: bool

In [ ]:
test1.isnull().any()

test_id              False
name                 False
item_condition_id    False
category_name         True
brand_name            True
shipping             False
item_description     False
dtype: bool

In [ ]:
test2.isnull().any()

test_id              False
name                 False
item_condition_id    False
category_name         True
brand_name            True
shipping             False
item_description      True
dtype: bool

Obs : We observed that missing value are present in category_name, brand_name and item_description. There is no missing value in Numerical Features.

In [ ]:
# How many missing values are present in each of them ?
print("Number of missing value in category_name : {} ({}%)".format(sum(train.category_name.isnull()), np.round(100*sum(train.category_name.isnull())/len(train.category_name),2)))
print("Number of missing value in brand_name : {} ({}%)".format(sum(train.brand_name.isnull()), np.round(100*sum(train.brand_name.isnull())/len(train.brand_name), 2) ))
print("Number of missing value in item_description : {} ({}%) ".format(sum(train.item_description.isnull()), np.round(100*sum(train.item_description.isnull())/ len(train.item_description),2) ))

Number of missing value in category_name : 6327 (0.43%)
Number of missing value in brand_name : 632682 (42.68%)
Number of missing value in item_description : 4 (0.0%) 


Obs : Number of missing value in brand_name in 632682 which is a 42.68% of the data.

In [ ]:
def fill_missing(df):
  """This funnction will fill missing value with appropriate label"""
  df.category_name.fillna(value = "others/others/others", inplace = True)
  df.brand_name.fillna(value = 'unknown ', inplace = True)
  df.item_description.fillna(value = "No description", inplace = True)
  return df

In [ ]:
train = fill_missing(train)
test1 = fill_missing(test1)
test2 = fill_missing(test2)

In [ ]:
train.isnull().any()

train_id             False
name                 False
item_condition_id    False
category_name        False
brand_name           False
price                False
shipping             False
item_description     False
dtype: bool

Spliting categorical feature

In [ ]:
def cat_split(text):
  """This function will take text catergory and will split it into three categories."""
  try:
    cat1, cat2, cat3 = text.split('/')
    return cat1, cat2, cat3
  except:
    return ("others", "others", "others")

In [ ]:
train['cat1'], train['cat2'], train['cat3'] = zip(*train['category_name'].apply(lambda x: cat_split(x)))
test1['cat1'], test1['cat2'], test1['cat3'] = zip(*test1['category_name'].apply(lambda x: cat_split(x)))
test2['cat1'], test2['cat2'], test2['cat3'] = zip(*test2['category_name'].apply(lambda x: cat_split(x)))
# train['category_name'].apply(lambda x: cat_split(x)) will return three value for each row. we are using unpacking operator '*'
# for making all rows data into three group.

## Data Cleaning of Text Feature

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# https://stackoverflow.com/a/47091490/4084039
import re
from tqdm import tqdm

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def clean(data):

  preprocessed_data = []
  for text in tqdm(data.values):
    #for sentance in tqdm(text.values):
    sent = decontracted(text)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = sent.replace('[rm]', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent) 
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords.words('english'))
    preprocessed_data.append(sent.lower().strip())
  return preprocessed_data

In [ ]:
train['brand_name'] = clean(train['brand_name'])
test1['brand_name'] = clean(test1['brand_name'])
test2['brand_name'] = clean(test2['brand_name'])

100%|██████████| 3460725/3460725 [08:52<00:00, 6502.35it/s]


In [ ]:
train['name'] = clean(train['name'])
test1['name'] = clean(test1['name'])
test2['name'] = clean(test2['name'])

100%|██████████| 3460725/3460725 [27:44<00:00, 2079.01it/s]


In [ ]:
train['item_description'] = clean(train['item_description'])
test1['item_description'] = clean(test1['item_description'])
test2['item_description'] = clean(test2['item_description'])

100%|██████████| 3460725/3460725 [2:41:41<00:00, 356.70it/s]


In [ ]:
import dill
file = 'gdrive/My Drive/Colab Notebooks/mercari_1.pkl'
dill.dump_session(file)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import dill
file = 'gdrive/My Drive/Colab Notebooks/mercari_1.pkl'
dill.load_session(file)

# Spliting data into train and cross validate

In [ ]:
print(train.columns)
print(test1.columns)
print(test2.columns)

Index(['train_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'price', 'shipping', 'item_description', 'cat1', 'cat2', 'cat3'],
      dtype='object')
Index(['test_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'shipping', 'item_description', 'cat1', 'cat2', 'cat3'],
      dtype='object')
Index(['test_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'shipping', 'item_description', 'cat1', 'cat2', 'cat3'],
      dtype='object')


In [ ]:
y_train = np.log(train["price"]+1)
train_ids = train['train_id'].values.astype(np.int32)
train.drop(['train_id', 'cat1', 'cat2', 'cat3'], axis=1, inplace=True)

test1_ids = test1['test_id'].values.astype(np.int32)
test1.drop(['test_id', 'cat1', 'cat2', 'cat3'], axis=1, inplace=True)

test2_ids = test2['test_id'].values.astype(np.int32)
test2.drop(['test_id', 'cat1', 'cat2', 'cat3'], axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(train, y_train, test_size=0.2, random_state=23)

# Feature Engineering

In [ ]:
# Adding Number of words of name features.
X_train['name_len'] = X_train['name'].str.len()
X_cv['name_len'] = X_cv['name'].str.len()
test1['name_len'] = test1['name'].str.len()
test2['name_len'] = test2['name'].str.len()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
train['name_len'].corr(y_train)

0.04716508203246952

In [ ]:
# Adding Number of words in Item Description
X_train['item_desc_len'] = X_train['item_description'].str.len()
X_cv['item_desc_len'] = X_cv['item_description'].str.len()
test1['item_desc_len'] = test1['item_description'].str.len()
test2['item_desc_len'] = test2['item_description'].str.len()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
X_train['item_desc_len'].corr(y_train)

0.033041513963906075

In [ ]:
# Giving 3 times more weightage to "name" and 7 times more weightage brand_name compare to item_description
X_train['text'] = X_train['name'] + ' ' + X_train['name'] + ' ' + X_train['name'] + ' ' + X_train['brand_name'] + ' ' + X_train['brand_name'] + ' ' + X_train['brand_name'] + ' ' + X_train['brand_name'] + ' ' + X_train['brand_name'] + ' ' + X_train['brand_name'] + ' ' + X_train['brand_name'] + ' ' + X_train['item_description']
X_cv['text'] = X_cv['name'] + ' ' + X_cv['name'] + ' ' + X_cv['name'] + ' ' + X_cv['brand_name'] + ' ' + X_cv['brand_name'] + ' ' + X_cv['brand_name'] + ' ' + X_cv['brand_name'] + ' ' + X_cv['brand_name'] + ' ' + X_cv['brand_name'] + ' ' + X_cv['brand_name'] + ' ' + X_cv['item_description']
test1['text'] = test1['name'] + ' ' + test1['name'] + ' ' + test1['name'] + ' ' + test1['brand_name'] + ' ' +test1['brand_name'] + ' ' +test1['brand_name'] + ' ' + test1['brand_name'] + ' ' + test1['brand_name'] + ' ' + test1['brand_name'] + ' ' + test1['brand_name'] + ' ' + test1['item_description']
test2['text'] = test2['name'] + ' ' + test2['name'] + ' ' + test2['name'] + ' ' + test2['brand_name'] + ' ' +test2['brand_name'] + ' ' +test2['brand_name'] + ' ' + test2['brand_name'] + ' ' + test2['brand_name'] + ' ' + test2['brand_name'] + ' ' + test2['brand_name'] + ' ' + test2['item_description']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# Adding count of numeric value in text
X_train['text_contain_num'] = X_train['text'].apply(lambda x : sum(y.isdigit() for y in x))
X_cv['text_contain_num'] = X_cv['text'].apply(lambda x : sum(y.isdigit() for y in x))
test1['text_contain_num'] = test1['text'].apply(lambda x : sum(y.isdigit() for y in x))
test2['text_contain_num'] = test2['text'].apply(lambda x : sum(y.isdigit() for y in x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
X_train['text_contain_num'].corr(y_train)

0.03833251480425795

In [ ]:
# Adding number of word in text feature
X_train['text_len'] = X_train['text'].str.len()
X_cv['text_len'] = X_cv['text'].str.len()
test1['text_len'] = test1['text'].str.len()
test2['text_len'] = test2['text'].str.len()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
X_train['text_len'].corr(y_train)

0.04648712628268135

In [ ]:
X_train.columns

Index(['name', 'item_condition_id', 'category_name', 'brand_name', 'price',
       'shipping', 'item_description', 'name_len', 'item_desc_len', 'text',
       'text_contain_num', 'text_len'],
      dtype='object')

In [ ]:
def features(array, column, row, method):
  try:
    return array[column][row]
  except:
    if method == 'mean':
      return np.mean(y_train)
    else :
      return np.median(y_train)

In [ ]:
np.mean(y_train)

2.9789133943491475

In [ ]:
X_train.columns

Index(['name', 'item_condition_id', 'category_name', 'brand_name', 'price',
       'shipping', 'item_description', 'name_len', 'item_desc_len', 'text',
       'text_contain_num', 'text_len'],
      dtype='object')

In [ ]:
mean_price_per_category = X_train.pivot_table('price', index='category_name', aggfunc='mean').sort_values(by='price',ascending=False)
median_price_per_category = X_train.pivot_table('price', index='category_name', aggfunc='median').sort_values(by='price',ascending=False)

mean_price_per_brand = X_train.pivot_table('price', index='brand_name', aggfunc='mean').sort_values(by='price',ascending=False)
median_price_per_brand = X_train.pivot_table('price', index='brand_name', aggfunc='median').sort_values(by='price',ascending=False)

mean_price_per_name_len = X_train.pivot_table('price', index='name_len', aggfunc='mean').sort_values(by='price',ascending=False)
median_price_per_name_len = X_train.pivot_table('price', index='name_len', aggfunc='median').sort_values(by='price',ascending=False)

mean_price_per_item_desc_len = X_train.pivot_table('price', index='item_desc_len', aggfunc='mean').sort_values(by='price',ascending=False)
median_price_per_item_desc_len = X_train.pivot_table('price', index='item_desc_len', aggfunc='median').sort_values(by='price',ascending=False)

mean_price_per_text_len = X_train.pivot_table('price', index='text_len', aggfunc='mean').sort_values(by='price',ascending=False)
median_price_per_text_len = X_train.pivot_table('price', index='text_len', aggfunc='median').sort_values(by='price',ascending=False)

mean_price_per_item_condition_id = X_train.pivot_table('price', index='item_condition_id', aggfunc='mean').sort_values(by='price',ascending=False)
median_price_per_item_condition_id = X_train.pivot_table('price', index='item_condition_id', aggfunc='median').sort_values(by='price',ascending=False)

In [ ]:
X_train['mean_price_per_category'] = X_train['category_name'].apply(lambda x : features(mean_price_per_category, 'price', x, 'mean'))
X_cv['mean_price_per_category'] = X_cv['category_name'].apply(lambda x : features(mean_price_per_category, 'price', x, 'mean'))
test1['mean_price_per_category'] = test1['category_name'].apply(lambda x : features(mean_price_per_category, 'price', x, 'mean'))
test2['mean_price_per_category'] = test2['category_name'].apply(lambda x : features(mean_price_per_category, 'price', x, 'mean'))

X_train['median_price_per_category'] = X_train['category_name'].apply(lambda x : features(median_price_per_category, 'price', x, 'median'))
X_cv['median_price_per_category'] = X_cv['category_name'].apply(lambda x : features(median_price_per_category, 'price', x, 'median'))
test1['median_price_per_category'] = test1['category_name'].apply(lambda x : features(median_price_per_category, 'price', x, 'median'))
test2['median_price_per_category'] = test2['category_name'].apply(lambda x : features(median_price_per_category, 'price', x, 'median'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [ ]:
X_train['mean_price_per_brand'] = X_train['brand_name'].apply(lambda x : features(mean_price_per_brand, 'price', x, 'mean'))
X_cv['mean_price_per_brand'] = X_cv['brand_name'].apply(lambda x : features(mean_price_per_brand, 'price', x, 'mean'))
test1['mean_price_per_brand'] = test1['brand_name'].apply(lambda x : features(mean_price_per_brand, 'price', x, 'mean'))
test2['mean_price_per_brand'] = test2['brand_name'].apply(lambda x : features(mean_price_per_brand, 'price', x, 'mean'))

X_train['median_price_per_brand'] = X_train['brand_name'].apply(lambda x : features(median_price_per_brand, 'price', x, 'median'))
X_cv['median_price_per_brand'] = X_cv['brand_name'].apply(lambda x : features(median_price_per_brand, 'price', x, 'median'))
test1['median_price_per_brand'] = test1['brand_name'].apply(lambda x : features(median_price_per_brand, 'price', x, 'median'))
test2['median_price_per_brand'] = test2['brand_name'].apply(lambda x : features(median_price_per_brand, 'price', x, 'median'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [ ]:
X_train['mean_price_per_name_len'] = X_train['name_len'].apply(lambda x : features(mean_price_per_name_len, 'price', x, 'mean'))
X_cv['mean_price_per_name_len'] = X_cv['name_len'].apply(lambda x : features(mean_price_per_name_len, 'price', x, 'mean'))
test1['mean_price_per_name_len'] = test1['name_len'].apply(lambda x : features(mean_price_per_name_len, 'price', x, 'mean'))
test2['mean_price_per_name_len'] = test2['name_len'].apply(lambda x : features(mean_price_per_name_len, 'price', x, 'mean'))

X_train['median_price_per_name_len'] = X_train['name_len'].apply(lambda x : features(median_price_per_name_len, 'price', x, 'median'))
X_cv['median_price_per_name_len'] = X_cv['name_len'].apply(lambda x : features(median_price_per_name_len, 'price', x, 'median'))
test1['median_price_per_name_len'] = test1['name_len'].apply(lambda x : features(median_price_per_name_len, 'price', x, 'median'))
test2['median_price_per_name_len'] = test2['name_len'].apply(lambda x : features(median_price_per_name_len, 'price', x, 'median'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [ ]:
X_train['mean_price_per_item_desc_len'] = X_train['item_desc_len'].apply(lambda x : features(mean_price_per_item_desc_len, 'price', x, 'mean'))
X_cv['mean_price_per_item_desc_len'] = X_cv['item_desc_len'].apply(lambda x : features(mean_price_per_item_desc_len, 'price', x, 'mean'))
test1['mean_price_per_item_desc_len'] = test1['item_desc_len'].apply(lambda x : features(mean_price_per_item_desc_len, 'price', x, 'mean'))
test2['mean_price_per_item_desc_len'] = test2['item_desc_len'].apply(lambda x : features(mean_price_per_item_desc_len, 'price', x, 'mean'))

X_train['median_price_per_item_desc_len'] = X_train['item_desc_len'].apply(lambda x : features(median_price_per_item_desc_len, 'price', x, 'median'))
X_cv['median_price_per_item_desc_len'] = X_cv['item_desc_len'].apply(lambda x : features(median_price_per_item_desc_len, 'price', x, 'median'))
test1['median_price_per_item_desc_len'] = test1['item_desc_len'].apply(lambda x : features(median_price_per_item_desc_len, 'price', x, 'median'))
test2['median_price_per_item_desc_len'] = test2['item_desc_len'].apply(lambda x : features(median_price_per_item_desc_len, 'price', x, 'median'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [ ]:
X_train['mean_price_per_text_len'] = X_train['text_len'].apply(lambda x : features(mean_price_per_text_len, 'price', x, 'mean'))
X_cv['mean_price_per_text_len'] = X_cv['text_len'].apply(lambda x : features(mean_price_per_text_len, 'price', x, 'mean'))
test1['mean_price_per_text_len'] = test1['text_len'].apply(lambda x : features(mean_price_per_text_len, 'price', x, 'mean'))
test2['mean_price_per_text_len'] = test2['text_len'].apply(lambda x : features(mean_price_per_text_len, 'price', x, 'mean'))

X_train['median_price_per_text_len'] = X_train['text_len'].apply(lambda x : features(median_price_per_text_len, 'price', x, 'median'))
X_cv['median_price_per_text_len'] = X_cv['text_len'].apply(lambda x : features(median_price_per_text_len, 'price', x, 'median'))
test1['median_price_per_text_len'] = test1['text_len'].apply(lambda x : features(median_price_per_text_len, 'price', x, 'median'))
test2['median_price_per_text_len'] = test2['text_len'].apply(lambda x : features(median_price_per_text_len, 'price', x, 'median'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [ ]:
X_train['mean_price_per_item_condition_id'] = X_train['item_condition_id'].apply(lambda x : features(mean_price_per_item_condition_id, 'price', x, 'mean'))
X_cv['mean_price_per_item_condition_id'] = X_cv['item_condition_id'].apply(lambda x : features(mean_price_per_item_condition_id, 'price', x, 'mean'))
test1['mean_price_per_item_condition_id'] = test1['item_condition_id'].apply(lambda x : features(mean_price_per_item_condition_id, 'price', x, 'mean'))
test2['mean_price_per_item_condition_id'] = test2['item_condition_id'].apply(lambda x : features(mean_price_per_item_condition_id, 'price', x, 'mean'))

X_train['median_price_per_item_condition_id'] = X_train['item_condition_id'].apply(lambda x : features(median_price_per_item_condition_id, 'price', x, 'median'))
X_cv['median_price_per_item_condition_id'] = X_cv['item_condition_id'].apply(lambda x : features(median_price_per_item_condition_id, 'price', x, 'median'))
test1['median_price_per_item_condition_id'] = test1['item_condition_id'].apply(lambda x : features(median_price_per_item_condition_id, 'price', x, 'median'))
test2['median_price_per_item_condition_id'] = test2['item_condition_id'].apply(lambda x : features(median_price_per_item_condition_id, 'price', x, 'median'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [ ]:
X_train.drop(['price'], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
X_train.columns

Index(['name', 'item_condition_id', 'category_name', 'brand_name', 'shipping',
       'item_description', 'name_len', 'item_desc_len', 'text',
       'text_contain_num', 'text_len', 'mean_price_per_category',
       'median_price_per_category', 'mean_price_per_brand',
       'median_price_per_brand', 'mean_price_per_name_len',
       'median_price_per_name_len', 'mean_price_per_item_desc_len',
       'median_price_per_item_desc_len', 'mean_price_per_text_len',
       'median_price_per_text_len', 'mean_price_per_item_condition_id',
       'median_price_per_item_condition_id'],
      dtype='object')

# One Hot Encoding Categorical Feature

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(X_train['category_name'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
X_train_cat = vectorizer.transform(X_train['category_name'].values)
X_cv_cat = vectorizer.transform(X_cv['category_name'].values)
test1_cat = vectorizer.transform(test1['category_name'].values)
test2_cat = vectorizer.transform(test2['category_name'].values)

print("After vectorizations")
print(X_train_cat.shape, y_train.shape)
print(X_cv_cat.shape, y_cv.shape)
print(test1_cat.shape)
print(test2_cat.shape)
print(vectorizer.get_feature_names())
print("="*1000)

After vectorizations
(1186028, 1007) (1186028,)
(296507, 1007) (296507,)
(693359, 1007)
(3460725, 1007)
['100', '24', '2t', '50', '5t', '75', 'above', 'accents', 'access', 'accessories', 'accessory', 'aceo', 'action', 'activity', 'adapters', 'adventure', 'advertisement', 'afghan', 'air', 'albums', 'all', 'amplifiers', 'and', 'animal', 'animals', 'animation', 'antique', 'apparel', 'appliances', 'applicators', 'apron', 'area', 'art', 'artist', 'arts', 'artwork', 'aspirators', 'asymmetrical', 'athletic', 'audio', 'automotive', 'baby', 'backpack', 'backpacks', 'badminton', 'bag', 'baggy', 'bags', 'baguette', 'bakeware', 'ballet', 'balls', 'band', 'bar', 'baseball', 'basic', 'basket', 'basketball', 'baskets', 'bass', 'bath', 'bathing', 'bathroom', 'batteries', 'beach', 'bead', 'beading', 'beads', 'bear', 'bears', 'beauty', 'bed', 'bedding', 'bedroom', 'beds', 'bedspreads', 'beer', 'belt', 'belts', 'beverage', 'bibles', 'bibs', 'bicycle', 'bike', 'binoculars', 'bins', 'biographies', 'biograp

# Bag of Words Representation of Text

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=20, ngram_range=(1, 4), max_features=20000)
vectorizer.fit(X_train['text'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
X_train_text_bow = vectorizer.transform(X_train['text'].values)
X_cv_text_bow = vectorizer.transform(X_cv['text'].values)
test1_text_bow = vectorizer.transform(test1['text'].values)
test2_text_bow = vectorizer.transform(test2['text'].values)

print("After vectorizations")
print(X_train_text_bow.shape, y_train.shape)
print(X_cv_text_bow.shape, y_cv.shape)
print(test1_text_bow.shape)
print(test2_text_bow.shape)
print(vectorizer.get_feature_names())
print("="*1000)

After vectorizations
(1186028, 20000) (1186028,)
(296507, 20000) (296507,)
(693359, 20000)
(3460725, 20000)
['00', '00 free', '00 free shipping', '000', '01', '01 product', '01 product detials', '02', '02 choose', '02 choose color', '02 choose color size', '03', '03 authentic', '04', '05', '06', '07', '08', '09', '0g', '10', '10 10', '10 11', '10 12', '10 13', '10 14', '10 black', '10 brand', '10 brand new', '10 business', '10 business days', '10 business days arrive', '10 condition', '10 free', '10 ft', '10 inches', '10 lululemon', '10 men', '10 new', '10 nike', '10 nike nike', '10 nike nike nike', '10 oz', '10 pairs', '10 pcs', '10 piece', '10 selling', '10 spandex', '10 unknown', '10 unknown unknown', '10 unknown unknown unknown', '10 women', '100', '100 authentic', '100 authentic brand', '100 authentic brand new', '100 authentic free', '100 authentic guaranteed', '100 brand', '100 brand new', '100 brand new high', '100 cotton', '100 guaranteed', '100 human', '100 human hair', '100 

# TFIDF Representation of Text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=20, ngram_range=(1, 4), max_features=20000)
vectorizer.fit(X_train['text'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
X_train_text_tfidf = vectorizer.transform(X_train['text'].values)
X_cv_text_tfidf = vectorizer.transform(X_cv['text'].values)
test1_text_tfidf = vectorizer.transform(test1['text'].values)
test2_text_tfidf = vectorizer.transform(test2['text'].values)

print("After vectorizations")
print(X_train_text_tfidf.shape, y_train.shape)
print(X_cv_text_tfidf.shape, y_cv.shape)
print(test1_text_tfidf.shape)
print(test2_text_tfidf.shape)
print(vectorizer.get_feature_names())
print("="*100)

After vectorizations
(1186028, 20000) (1186028,)
(296507, 20000) (296507,)
(693359, 20000)
(3460725, 20000)
['00', '00 free', '00 free shipping', '000', '01', '01 product', '01 product detials', '02', '02 choose', '02 choose color', '02 choose color size', '03', '03 authentic', '04', '05', '06', '07', '08', '09', '0g', '10', '10 10', '10 11', '10 12', '10 13', '10 14', '10 black', '10 brand', '10 brand new', '10 business', '10 business days', '10 business days arrive', '10 condition', '10 free', '10 ft', '10 inches', '10 lululemon', '10 men', '10 new', '10 nike', '10 nike nike', '10 nike nike nike', '10 oz', '10 pairs', '10 pcs', '10 piece', '10 selling', '10 spandex', '10 unknown', '10 unknown unknown', '10 unknown unknown unknown', '10 women', '100', '100 authentic', '100 authentic brand', '100 authentic brand new', '100 authentic free', '100 authentic guaranteed', '100 brand', '100 brand new', '100 brand new high', '100 cotton', '100 guaranteed', '100 human', '100 human hair', '100 

# Standardized Numerical Feature

In [ ]:
X_train.columns

Index(['name', 'item_condition_id', 'category_name', 'brand_name', 'shipping',
       'item_description', 'name_len', 'item_desc_len', 'text',
       'text_contain_num', 'text_len', 'mean_price_per_category',
       'median_price_per_category', 'mean_price_per_brand',
       'median_price_per_brand', 'mean_price_per_name_len',
       'median_price_per_name_len', 'mean_price_per_item_desc_len',
       'median_price_per_item_desc_len', 'mean_price_per_text_len',
       'median_price_per_text_len', 'mean_price_per_item_condition_id',
       'median_price_per_item_condition_id'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
ss = scaler.fit(X_train[['name_len', 'item_desc_len', 'text_contain_num', 'text_len', 'mean_price_per_category', 'median_price_per_category', 
                         'mean_price_per_brand', 'median_price_per_brand', 'mean_price_per_name_len', 'median_price_per_name_len',
                         'mean_price_per_item_desc_len', 'median_price_per_item_desc_len', 'mean_price_per_text_len', 'median_price_per_text_len',
                         'mean_price_per_item_condition_id', 'median_price_per_item_condition_id']])
X_train_numerical = scaler.transform(X_train[['name_len', 'item_desc_len', 'text_contain_num', 'text_len', 'mean_price_per_category', 'median_price_per_category', 
                         'mean_price_per_brand', 'median_price_per_brand', 'mean_price_per_name_len', 'median_price_per_name_len',
                         'mean_price_per_item_desc_len', 'median_price_per_item_desc_len', 'mean_price_per_text_len', 'median_price_per_text_len',
                         'mean_price_per_item_condition_id', 'median_price_per_item_condition_id']])
X_cv_numerical = scaler.transform(X_cv[['name_len', 'item_desc_len', 'text_contain_num', 'text_len', 'mean_price_per_category', 'median_price_per_category', 
                         'mean_price_per_brand', 'median_price_per_brand', 'mean_price_per_name_len', 'median_price_per_name_len',
                         'mean_price_per_item_desc_len', 'median_price_per_item_desc_len', 'mean_price_per_text_len', 'median_price_per_text_len',
                         'mean_price_per_item_condition_id', 'median_price_per_item_condition_id']])
test1_numerical = scaler.transform(test1[['name_len', 'item_desc_len', 'text_contain_num', 'text_len', 'mean_price_per_category', 'median_price_per_category', 
                         'mean_price_per_brand', 'median_price_per_brand', 'mean_price_per_name_len', 'median_price_per_name_len',
                         'mean_price_per_item_desc_len', 'median_price_per_item_desc_len', 'mean_price_per_text_len', 'median_price_per_text_len',
                         'mean_price_per_item_condition_id', 'median_price_per_item_condition_id']])
test2_numerical = scaler.transform(test2[['name_len', 'item_desc_len', 'text_contain_num', 'text_len', 'mean_price_per_category', 'median_price_per_category', 
                         'mean_price_per_brand', 'median_price_per_brand', 'mean_price_per_name_len', 'median_price_per_name_len',
                         'mean_price_per_item_desc_len', 'median_price_per_item_desc_len', 'mean_price_per_text_len', 'median_price_per_text_len',
                         'mean_price_per_item_condition_id', 'median_price_per_item_condition_id']])

print(X_train_numerical.shape)
print(X_cv_numerical.shape)
print(test1_numerical.shape)
print(test2_numerical.shape)

(1186028, 16)
(296507, 16)
(693359, 16)
(3460725, 16)


In [ ]:
from scipy.sparse import csr_matrix
X_train_dummies = pd.get_dummies(X_train[['item_condition_id', 'shipping']])
X_cv_dummies = pd.get_dummies(X_cv[['item_condition_id', 'shipping']])
test1_dummies = pd.get_dummies(test1[['item_condition_id', 'shipping']])
test2_dummies = pd.get_dummies(test2[['item_condition_id', 'shipping']])

print(X_train_dummies.shape)
print(X_cv_dummies.shape)
print(test1_dummies.shape)
print(test2_dummies.shape)

(1186028, 2)
(296507, 2)
(693359, 2)
(3460725, 2)


In [ ]:
from scipy.sparse import hstack
X_train_bow = hstack((X_train_cat, X_train_text_bow, X_train_numerical, X_train_dummies)).tocsr()
X_cv_bow = hstack((X_cv_cat, X_cv_text_bow, X_cv_numerical, X_cv_dummies)).tocsr()
test1_bow = hstack((test1_cat, test1_text_bow, test1_numerical, test1_dummies)).tocsr()
test2_bow = hstack((test2_cat, test2_text_bow, test2_numerical, test2_dummies)).tocsr()

print("Final Data matrix")
print(X_train_bow.shape, y_train.shape)
print(X_cv_bow.shape, y_cv.shape)
print(test1_bow.shape)
print(test2_bow.shape)

Final Data matrix
(1186028, 21025) (1186028,)
(296507, 21025) (296507,)
(693359, 21025)
(3460725, 21025)


In [ ]:
import joblib
joblib.dump(X_train_bow, '/content/gdrive/My Drive/Mercari/X_train_bow1.pkl')
joblib.dump(X_cv_bow, '/content/gdrive/My Drive/Mercari/X_cv_bow1.pkl')
joblib.dump(test1_bow, '/content/gdrive/My Drive/Mercari/test1_bow1.pkl')
joblib.dump(test2_bow, '/content/gdrive/My Drive/Mercari/test2_bow1.pkl')

In [ ]:
from scipy.sparse import hstack
X_train_tfidf = hstack((X_train_cat, X_train_text_tfidf, X_train_numerical, X_train_dummies)).tocsr()
X_cv_tfidf = hstack((X_cv_cat, X_cv_text_tfidf, X_cv_numerical, X_cv_dummies)).tocsr()
test1_tfidf = hstack((test1_cat, test1_text_tfidf, test1_numerical, test1_dummies)).tocsr()
test2_tfidf = hstack((test2_cat, test2_text_tfidf, test2_numerical, test2_dummies)).tocsr()

print("Final Data matrix")
print(X_train_tfidf.shape, y_train.shape)
print(X_cv_tfidf.shape, y_cv.shape)
print(test1_tfidf.shape)
print(test2_tfidf.shape)

Final Data matrix
(1186028, 21025) (1186028,)
(296507, 21025) (296507,)
(693359, 21025)
(3460725, 21025)


In [ ]:
joblib.dump(X_train_tfidf, '/content/gdrive/My Drive/Mercari/X_train_tfidf1.pkl')
joblib.dump(X_cv_tfidf, '/content/gdrive/My Drive/Mercari/X_cv_tfidf1.pkl')
joblib.dump(test1_tfidf, '/content/gdrive/My Drive/Mercari/test1_tfidf1.pkl')
joblib.dump(test2_tfidf, '/content/gdrive/My Drive/Mercari/test2_tfidf1.pkl')

In [ ]:
joblib.dump(y_train, '/content/gdrive/My Drive/Mercari/y_train.pkl')
joblib.dump(y_cv, '/content/gdrive/My Drive/Mercari/y_cv.pkl')

# Loading Data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import joblib

X_train_bow = joblib.load('/content/gdrive/My Drive/Mercari/X_train_bow1.pkl')
X_cv_bow = joblib.load('/content/gdrive/My Drive/Mercari/X_cv_bow1.pkl')
test1_bow = joblib.load('/content/gdrive/My Drive/Mercari/test1_bow1.pkl')
test2_bow = joblib.load('/content/gdrive/My Drive/Mercari/test2_bow1.pkl')

y_train = joblib.load('/content/gdrive/My Drive/Mercari/y_train.pkl')
y_cv = joblib.load('/content/gdrive/My Drive/Mercari/y_cv.pkl')

In [ ]:
X_train_tfidf = joblib.load('/content/gdrive/My Drive/Mercari/X_train_tfidf1.pkl')
X_cv_tfidf = joblib.load('/content/gdrive/My Drive/Mercari/X_cv_tfidf1.pkl')
test1_tfidf = joblib.load('/content/gdrive/My Drive/Mercari/test1_tfidf1.pkl')
test2_tfidf = joblib.load('/content/gdrive/My Drive/Mercari/test2_tfidf1.pkl')

# Modeling

In [ ]:
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
#from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
parameters = {"alpha":[0.00000001,0.00001,0.0001,0.001,0.01,0.1,0,1,10,100,1000,10000,100000]}
ridgeReg = Ridge(solver = "lsqr", fit_intercept=False)
lr_reg = GridSearchCV(ridgeReg,param_grid =parameters,n_jobs=-1)
lr_reg.fit(X_train_bow, y_train)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GridSearchCV(cv=None, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=False,
                             max_iter=None, normalize=False, random_state=None,
                             solver='lsqr', tol=0.001),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [1e-08, 1e-05, 0.0001, 0.001, 0.01, 0.1, 0, 1,
                                   10, 100, 1000, 10000, 100000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
lr_reg.best_params_

{'alpha': 1}

In [ ]:
ridgeReg = Ridge(alpha=1,solver = "lsqr", fit_intercept=False )
ridgeReg.fit(X_train_bow, y_train)
y_pred = ridgeReg.predict(X_cv_bow)

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
ridge_RMSLE = np.sqrt(mean_squared_error(y_cv, y_pred))
print('Fold %02d Ridge RMSLE: ', ridge_RMSLE)

 Fold %02d Ridge RMSLE:  0.5167011574371783


In [ ]:
parameters = {"alpha":[0.00000001,0.00001,0.0001,0.001,0.01,0.1,0,1,10,100,1000,10000,100000]}
ridgeReg = Ridge(solver = "lsqr", fit_intercept=False)
lr_reg = GridSearchCV(ridgeReg,param_grid =parameters,n_jobs=-1)
lr_reg.fit(X_train_tfidf, y_train)

In [ ]:
lr_reg.best_params_

In [ ]:
ridgeReg = Ridge(alpha=1,solver = "lsqr", fit_intercept=False )
ridgeReg.fit(X_train_tfidf, y_train)
y_pred = ridgeReg.predict(X_cv_tfidf)

In [ ]:
from sklearn.metrics import mean_squared_error
ridge_RMSLE = np.sqrt(mean_squared_error(y_cv, y_pred))
print('Fold %02d Ridge RMSLE : ', ridge_RMSLE)

Fold %02d Ridge RMSLE :  0.5173728077875336


In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=.1)
clf.fit(X_train_bow, y_train)
y_pred = clf.predict(X_cv_bow)

In [ ]:
lasso_RMSLE = np.sqrt(mean_squared_error(y_cv, y_pred))
print('Fold %02d Ridge RMSLE : ', lasso_RMSLE)

Fold %02d Ridge RMSLE :  0.6379807252381352


In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
lgbm_params ={'subsample': 0.9, 'colsample_bytree': 0.8, 'min_child_samples': 50, 'objective': 'regression','boosting_type': 'gbdt','learning_rate': 0.5,'max_depth': 8,'n_estimators': 500,'num_leaves': 80}
model = LGBMRegressor(**lgbm_params)
model.fit(X_train_bow, y_train, verbose=True)

print('Generating submission...')
preds2 = model.predict(X_cv_bow)

Generating submission...


In [ ]:
lgbm_RMSLE = np.sqrt(mean_squared_error(y_cv, preds2))
print('Fold %02d Ridge RMSLE: ', lgbm_RMSLE)

Fold %02d Ridge RMSLE:  0.4587382924466272


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor, Ridge

In [ ]:
model = SGDRegressor(penalty='l2', loss='squared_loss', learning_rate='invscaling', max_iter=200, fit_intercept=False, alpha=0, l1_ratio=0.6)
model.fit(X_train_bow, y_train)

SGDRegressor(alpha=0, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=False, l1_ratio=0.6,
             learning_rate='invscaling', loss='squared_loss', max_iter=200,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [ ]:
sgd_scores_val = model.predict(X_cv_bow)
sgd_RMSLE = np.sqrt(mean_squared_error(y_cv, sgd_scores_val))
print(sgd_RMSLE)

0.5116107466885108


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [ ]:
m = RandomForestRegressor(n_jobs=-1,min_samples_split=3,n_estimators=200,max_depth=40)
m.fit(X_train_bow, y_train)

In [ ]:
predr = m.predict(X_cv_bow)
rf_RMSLE = np.sqrt(mean_squared_error(y_test, predr))
print(rf_RMSLE)